In [1]:
import numpy as np
import pandas as pd
import tqdm
import barcodes_select

If the error rate is 2% and the length of a barcode is 9bp, we will be able to detect 97.39% of the data.

In [10]:
from scipy.stats import binom
binom.cdf(1, 9, 0.02)**2

0.9739422128292623

Read potential barcodes as list.

In [3]:
from functools import reduce

ini_lista = pd.read_csv('indexa.csv',header = None).values.tolist()
indexa = reduce(lambda z, y :z + y, ini_lista)
indexa = [x for x in indexa if str(x) != 'nan']
 
ini_listb = pd.read_csv('indexb.csv',header = None).values.tolist()
indexb = reduce(lambda z, y :z + y, ini_listb)

Accurancy array. Got from [here](https://doi.org/10.1101/2023.03.29.534691) in nanopore direct RNA sequencing. Accuracy for R10.4 should be much hiher.

The array is a 4*4 probability matrix. The row should be original ’ATCG‘ and the called ’ATCG‘.

In [4]:
accurancy = np.array([[0.923, 0.013, 0.004, 0.010],
                         [0.016, 0.887, 0.021, 0.004],
                         [0.009, 0.029, 0.893, 0.002],
                         [0.017, 0.005, 0.002, 0.944],
                        ])
accurancy.sum(axis = 1),accurancy.sum(axis = 0)

(array([0.95 , 0.928, 0.933, 0.968]), array([0.965, 0.934, 0.92 , 0.96 ]))

We define a barcode class here, imputs are barcodelist, barcode_num (target barcode number), tolerance (the number of error tolerance, normally 1), accurancy_matrix.

In [5]:
barb = barcodes_select.Barcodes(indexb, barcode_num = 50, tolerance = 1, accurancy_matrix = accurancy)

100%|██████████| 384/384 [04:58<00:00,  1.29it/s]


Final result

In [6]:
barb.prob_barcodes()

array(['AAACCATAG', 'AGAGCATGT', 'ATGGTAACT', 'AAGTACGTT', 'CGACGATAT',
       'TAGAGAGTA', 'TCAAGATCT', 'ATACTACTC', 'CATCAACGT', 'TGCCTATTA',
       'ACAACCTAT', 'AATACCGAA', 'ATTCCTAGA', 'ACAGGTATT', 'ACTGAATAC',
       'AAGGATATG', 'TAGCCAATT', 'GGTACGGAT', 'GTAAGGAGT', 'ATTCGCAAT',
       'CCTGCAGTT', 'TCGTTCTGT', 'AACGATCAT', 'TGGTTGAAT', 'ATCAATACG',
       'CTGAGCATT', 'AACTAGTTG', 'ACATGGTAA', 'TGAATCTGA', 'ACTTATGGT',
       'GTATCGCAT', 'TCAGTTGGT', 'TCTAGGAAT', 'GGACTTGAT', 'CAACTCTCT',
       'TATAAGAGG', 'AACGTAATC', 'TAGAACCAA', 'TGTTAAGAC', 'TATCATGAG',
       'CTGATAGGT', 'ACCTCAATA', 'AGAAGTAAG', 'GTCGGAGTT', 'TTATAGGCA',
       'GAGACTAGT', 'GAGGAACTT', 'AGTTAACCA', 'TGAGAATCA', 'TTCCATTAC'],
      dtype='<U9')

The largest probability for one barcode match to another.

In [7]:
barb.final_prob_matirx.max()

1.5993734430403105e-08

Do the same stuff for 12 target barcodes.

In [8]:
barb12 = barcodes_select.Barcodes(indexb, barcode_num = 12, tolerance = 1, accurancy_matrix = accurancy)
print(barb12.prob_barcodes())
print(barb12.final_prob_matirx.max())

100%|██████████| 384/384 [05:02<00:00,  1.27it/s]


['AAACCATAG' 'TAGAGAGTA' 'AATACCGAA' 'ATTCGCAAT' 'TGGTTGAAT' 'ACATGGTAA'
 'TATAAGAGG' 'AACGTAATC' 'AGAAGTAAG' 'GAGACTAGT' 'TGAGAATCA' 'TTCCATTAC']
4.959724692001997e-11


Do the same stuff for 50 target barcodes in the first round.

In [9]:
bara = barcodes_select.Barcodes(indexa, barcode_num = 50, tolerance = 1, accurancy_matrix = accurancy)
print(bara.prob_barcodes())
print(bara.final_prob_matirx.max())

100%|██████████| 96/96 [00:24<00:00,  3.91it/s]

['GCGTTGGAGC' 'GATCTTACGC' 'CCGAGAATCC' 'GCCGCAACGA' 'TGCGGACCTA'
 'ACGGAGGCGG' 'TTATTCATTC' 'AGAGCTATAA' 'CTAAGAGAAG' 'GGTACTGCCT'
 'TGCCGGCAGA' 'TTACCGAGGC' 'ACTATGCAAT' 'CGACGCGACT' 'GATACGGAAC'
 'TAGAGTAATA' 'TCGGCCTTAC' 'AGAACGTCTC' 'ACTTAACCTT' 'GAAGATCGAG'
 'AAGAAGCTAG' 'TCCGGCCTCG' 'AGAGAAGGTT' 'GCTAACTTGC' 'GGCTGAGCTC'
 'ATAAGGAGCA' 'GGTATGCTTG' 'TAGCCGTCAT' 'CTAGTAGTCT' 'AACTAGGCGC'
 'TCGCTAAGCA' 'TATATACTAA' 'AACCATTGGA' 'TCGCGGTTGG' 'CGTAGTTACC'
 'AATCGATAAT' 'CCATTATCTA' 'TCAACGTAAG' 'CTAACTAGAT' 'CATTCAATCA'
 'ATCGGCTATC' 'GGAGGATAGC' 'GGCTCTCTAT' 'CGCTCCTAAC' 'TCTTGCCGAC'
 'AGGTTAGCAT' 'TTCGCCTCCA' 'CATCTCTGCA' 'ACCTGGCCAA' 'TAACTGGTTA']
8.257759737968713e-10
